# AI-AvisaFallah-Assignment2

## Goal
In this assignment, we're going to formulate a searching problem and try to solve that using uninformed(DFS, BFS, UCS and IDS) and informed(A*) algorithms.


in this part of code we define the board of the 8-puzzle game which all these board classes will get togheter and create a tree which would be the solution path

In [9]:
import numpy as np

class Board:
    parent = None
    state = None
    operator = None
    depth = 0
    zero = None
    cost = 0
    final = np.array([1,2,3,4,5,6,7,8,0])

    #constructor which we dfine parent curr state operators depth cost and goal state
    def __init__(self, state, parent=None, operator=None, depth=0):
        self.parent = parent
        self.state = np.array(state)
        self.operator = operator
        self.depth = depth
        self.zero = self.find_0()
        self.cost = self.depth + self.manhattan()
        self.final = np.array([1,2,3,4,5,6,7,8,0])
    # less than operator for A*
    def __lt__(self, other):
        if self.cost != other.cost:
            return self.cost < other.cost
        else:
            op_pr = {'Up': 0, 'Down': 1, 'Left': 2, 'Right': 3}
            return op_pr[self.operator] < op_pr[other.operator]


    def __str__(self):
        return str(self.state[:3]) + '\n' \
               + str(self.state[3:6]) + '\n' \
               + str(self.state[6:]) + ' ' + str(self.depth) + str(self.operator) + '\n'

    # our goal test function
    def goal_test(self):
        if np.array_equal(self.state, self.final):
            return True
        else:
            return False

    #find index of first 0
    def find_0(self):
        for i in range(9):
            if self.state[i] == 0:
                return i

    # Heuristic function for A*
    def manhattan(self):
        state = self.index(self.state)
        goal = self.index(self.final)
        return sum((abs(state // 3 - goal // 3) + abs(state % 3 - goal % 3))[1:])

    # return indicies
    @staticmethod
    def index(state):
        index = np.array(range(9))
        for x, y in enumerate(state):
            index[y] = x
        return index

    #swaping two elements
    def swap(self, i, j):
        new_state = np.array(self.state)
        new_state[i], new_state[j] = new_state[j], new_state[i]
        return new_state

    # move up
    def up(self):
        if self.zero > 2:
            return Board(self.swap(self.zero, self.zero - 3), self, 'Up', self.depth + 1)
        else:
            return None

    # move down
    def down(self):
        if self.zero < 6:
            return Board(self.swap(self.zero, self.zero + 3), self, 'Down', self.depth + 1)
        else:
            return None

    #move left
    def left(self):
        if self.zero % 3 != 0:
            return Board(self.swap(self.zero, self.zero - 1), self, 'Left', self.depth + 1)
        else:
            return None

    # move right
    def right(self):
        if (self.zero + 1) % 3 != 0:
            return Board(self.swap(self.zero, self.zero + 1), self, 'Right', self.depth + 1)
        else:
            return None

    # find list of neighbors
    def neighbors(self):
        neighbors = [self.up(), self.down(), self.left(), self.right()]
        return list(filter(None, neighbors))

    __repr__ = __str__

# Informed Algorithms
## A*
The idea behind this algorithm is to estimate remaining cost from each state to goal state and then in each step, try to expand the node with smallest overall cost(cost to get to state + estimated remaining cost to goal state). We call this estimation, the heuristic function for A* algorithm. 

In this problem, we use Manhattan distance as heuristic function


A good heuristic function is a function that doesn't overestimate paths' costs. This property is known as admissiblity and consistency. An admissible heuristic's estimation for each state is less or equal to true cost to reach goal from that state. In a consistent heuristic, this inequality holds:

$$cost(A to C) \geq h(A) - h(C)$$, where A and C are two states.

As you can see, our heuristic function doesn't overestimate costs(since in each state, the minimum true cost is what our function estimates and thus, this function is admissible. It's easy to show that heuristic function is consistent too. We show check inquality mentioned above for each two possible neighbour states.

So let's implement this algorithm. There is a challenge in implementing this algorithm: We need a data structure to store open states to get the minimum f value(f = g(current cost) + h) in optimal time, which is usually implemented using priority queue.
We first move the empty space in all the possible directions in the start state and calculate the f-score for each state. This is called expanding the current state.
After expanding the current state, it is pushed into the closed list and the newly generated states are pushed into the open list. A state with the least f-score is selected and expanded again. This process continues until the goal state occurs as the current state. Basically, here we are providing the algorithm a measure to choose its actions. The algorithm chooses the best possible action and proceeds in that path.
This solves the issue of generating redundant child states, as the algorithm will expand the node with the least f-score

## Pros
- It's complete(assuming number of nodes with f(n) <= C* is finite).
- It's optimal(since cost is 1 per step).
- This is the best one of all the other algorithms if we choose a good heuristic..
- The algorithm is optimally efficient, i.e., there is no other optimal algorithm that is guaranteed to expand fewer nodes than A*.

## Cons
- The speed execution of A* search is highly dependant on the accuracy of the heuristic algorithm that is used to compute h(n) and implementation and is a bit slower than algorithms like BFS.
- It's sometimes difficult to define a good heuristic function.

In [3]:
import heapq

class AStar():
    solution = None
    frontier = None
    nodes_expanded = 0
    max_depth = 0
    explored_nodes = set()
    initial_state = None

    def __init__(self, initial_state):
          self.explored_nodes.clear()
          self.initial_state = initial_state
          self.frontier = []

    def ancestral_chain(self):
        current = self.solution
        chain = [current]
        while current.parent is not None:
            chain.append(current.parent)
            current = current.parent
        return chain

    def path(self):
        path = [node.operator for node in self.ancestral_chain()[-2::-1]]
        return path

    def set_solution(self, board):
        self.solution = board
        self.nodes_expanded = len(self.explored_nodes) - len(self.frontier) - 1
        # return self.solution

    def solve(self):
        # automatically sorts by depth + manhattan(defined __lt__ in board class)
        heapq.heappush(self.frontier, self.initial_state)
        while self.frontier:
            # pop the best cost = depth + manhattan
            board = heapq.heappop(self.frontier)
            new_board = np.append(board.state, board.depth)
            current_state = tuple(new_board)
            self.explored_nodes.add(current_state)
            np.delete(new_board, -1)
            # goal test
            if board.goal_test():
                self.set_solution(board)
                break
            # expand all neighbors of that node
            for neighbor in board.neighbors():
                new_neighbor = np.append(neighbor.state, neighbor.depth)
                # check if current neighbor is not in explored
                if tuple(new_neighbor) not in self.explored_nodes:
                    heapq.heappush(self.frontier, neighbor)
                    self.explored_nodes.add(tuple(new_neighbor))
                    self.max_depth = max(self.max_depth, neighbor.depth)
        return

# Uninformed Algorithms
## UCS
Uniform-cost search is an uninformed search algorithm that uses the lowest cumulative cost to find a path from the source to the destination. Nodes are expanded, starting from the root, according to the minimum cumulative cost. The uniform-cost search is then implemented using a Priority Queue.

In [4]:
import heapq

class UCS():
    solution = None
    frontier = None
    nodes_expanded = 0
    max_depth = 0
    explored_nodes = set()
    initial_state = None

    def __init__(self, initial_state):
          self.explored_nodes.clear()
          self.initial_state = initial_state
          self.frontier = []

    def ancestral_chain(self):
        current = self.solution
        chain = [current]
        while current.parent is not None:
            chain.append(current.parent)
            current = current.parent
        return chain

    def path(self):
        path = [node.operator for node in self.ancestral_chain()[-2::-1]]
        return path

    def set_solution(self, board):
        self.solution = board
        self.nodes_expanded = len(self.explored_nodes) - len(self.frontier) - 1
        # return self.solution

    def check_exists(self, my_tup):
        for i in range(0, len(self.frontier)):
            # print(self.frontier[i])
            if np.array_equal(self.frontier[i][1].state, my_tup[1].state):
                #print(self.frontier[i][0], my_tup[0])
                #print(self.frontier[i][0], my_tup[0])
                min_var = min(self.frontier[i][0], my_tup[0])
                self.frontier.remove(self.frontier[i])
                return True, min_var


        return False, 0

    def solve(self):
        # sort heapq by (self.initial_state.depth)
        heapq.heappush(self.frontier, ((self.initial_state.depth), self.initial_state))
        while self.frontier:
            # pop the least depth
            node = heapq.heappop(self.frontier)
            board = node[1]
            current_state = tuple(board.state)
            # push in explored Set
            self.explored_nodes.add(current_state)
            # check if reached to goal state
            if board.goal_test():
                self.set_solution(board)
                break
            # expand all neighbors of that node
            for neighbor in board.neighbors():
                # check if current neighbor is not in explored and frontier
                exists, min_depth = self.check_exists((neighbor.depth, neighbor))
                if tuple(neighbor.state) not in self.explored_nodes and not exists:
                    heapq.heappush(self.frontier, (neighbor.depth, neighbor))
                elif exists:
                    heapq.heapify(self.frontier)
                    heapq.heappush(self.frontier, (min_depth, neighbor))


        return

## BFS
In BFS, frontier set acts like a FIFO queue and in each step, we'll expand the shallowest state in the frontier set and add the state to the explored set, which is used to avoid duplicate checking for states. Pay attention! We use explored set for states that are in frontier set and not explored yet too. This is because searching in set is much faster than searching in list(because it's implemented by hash table) and this will make the algorithm faster. We'll keep track of total states checked and traversal tree during execution.

Breadth-first search is one of the simplest algorithms for searching a graph, it expands the nodes in a tree in the order of their given distance from the root, so it expands all the neighbouring nodes before going to the next level of the tree. The algorithm doesn’t trawl to the deeper levels of the tree without first expanding the lower levels thus ensures the finding of the shortest path.

The space requirement of breadth-first search is its largest deficiency. The 8-tile has a search space of 9!/2 = 181,400 states with a maximum number of 31 moves to solve.

## Pros
- It's complete(assuming branching factor is finite).
- It's optimal(since cost is 1 per step).
- Time spent isn't bad for an uninformed search algorithm.

## Cons
- Memory usage is high(since we keep track of many states per step).
- For far goal states, consumed time is high.

In [5]:
from collections import deque

class BFS():
    solution = None
    frontier = None
    nodes_expanded = 0
    max_depth = 0
    explored_nodes = set()
    initial_state = None

    def __init__(self, initial_state):
          self.explored_nodes.clear()
          self.initial_state = initial_state
          self.frontier = deque()

    def ancestral_chain(self):
        current = self.solution
        chain = [current]
        while current.parent is not None:
            chain.append(current.parent)
            current = current.parent
        return chain

    def path(self):
        path = [node.operator for node in self.ancestral_chain()[-2::-1]]
        return path

    def set_solution(self, board):
        self.solution = board
        self.nodes_expanded = len(self.explored_nodes) - len(self.frontier) - 1
        # return self.solution
      

    def solve(self):
        self.frontier.append(self.initial_state)
        while self.frontier:
            board = self.frontier.popleft()
            new_board = np.append(board.state, board.depth)
            current_state = tuple(new_board)
            self.explored_nodes.add(current_state)
            np.delete(new_board, -1)
            if board.goal_test():
                self.set_solution(board)
                break
            for neighbor in board.neighbors():
                new_neighbor = np.append(neighbor.state, neighbor.depth)
                if tuple(new_neighbor) not in self.explored_nodes:
                    self.frontier.append(neighbor)
                    self.explored_nodes.add(tuple(new_neighbor))
                    self.max_depth = max(self.max_depth, neighbor.depth)
        return

## DFS
Depth First Search (DFS) starts at a node and proceeds down the left-most node until it reaches a leaf. It then backs up to the leaf’s parent and checks it next left-most node, and so on.
DFS is also a complete solution that will ultimately find the goal if it exists.

It should be noted that searching graphs is a different challenge than searching trees. With trees, you can be certain that your search will not enter into any cycles (loops), but in a graph, nodes can cycle back to itself, which is a problem. To prevent that you simply track whether a node has been visited or not (it's pretty simple).

DFS excels if the goal is located in one of the far left branches in the graph. However, if the goal is shallowly located in one of the right-most branches it might take some time to get there. 

# Pros:
- The memory requirement is Linear WRT Nodes.
- Less time and space complexity rather than BFS.
- The solution can be found out without much more search.

#Cons
- Not Guaranteed that it will give you a solution.
- Cut-off depth is smaller so time complexity is more.
- Determination of depth until the search has proceeded.

In [6]:
class DFS():
    solution = None
    frontier = None
    nodes_expanded = 0
    max_depth = 0
    explored_nodes = set()
    initial_state = None
    limit_depth = 32
    def __init__(self, initial_state, limit_depth):
          self.explored_nodes.clear()
          self.initial_state = initial_state
          self.frontier = []
          self.limit_depth = limit_depth


    def ancestral_chain(self):
        current = self.solution
        chain = [current]
        while current.parent is not None:
            chain.append(current.parent)
            current = current.parent
        return chain

    def path(self):
        path = [node.operator for node in self.ancestral_chain()[-2::-1]]
        return path

    def set_solution(self, board):
        self.solution = board
        self.nodes_expanded = len(self.explored_nodes) - len(self.frontier) - 1
        # return self.solution
        
    def solve(self):
        self.frontier.append(self.initial_state)
        while self.frontier:
            board = self.frontier.pop()
            current_depth = board.depth
            new_board = np.append(board.state, board.depth)
            current_state = tuple(new_board)
            self.explored_nodes.add(current_state)
            np.delete(new_board, -1)
            if board.goal_test():
                self.set_solution(board)
                break
            if(current_depth < self.limit_depth):
                #print("!!!!!!!!!!neigbors:",board.neighbors())
                for neighbor in board.neighbors()[::-1]:
                    new_neighbor = np.append(neighbor.state, neighbor.depth)
                    #print(new_neighbor)
                    if tuple(new_neighbor) not in self.explored_nodes:
                        self.frontier.append(neighbor)
                        self.explored_nodes.add(tuple(new_neighbor))
                        self.max_depth = max(self.max_depth, neighbor.depth)
        
        return

## IDS
In IDS, we use DFS algorithm with limited depth and use an iterative approach to check several depth to find the goal state. Frontier set acts like a LIFO queue and in each step, we'll expand the deepest state in the frontier set and add the state and its current depth to the explored set, which is used to avoid duplicate checking for states. Pay attention! We'll add a duplicate state if current depth is less than previously seen depth(since a new path is found with less shorter length). We'll reset these sets if we couldn't find a path in a specific depth. We'll keep track of total states checked and traversal tree during execution of algorithm.

## Pros
- It's complete(assuming branching factor is finite).
- It's optimal(since cost is 1 per step).
- If goal state is found at the lower depths, then the algorithm proves to be efficient.
- Memory usage is good(if we free memory after each iteration) since it's linear.

## Cons
- Consumed time is really high(since we search at many depths)
- It calculates initial level too many times. 

In [32]:
import itertools

class IDS():
    solution = None
    frontier = None
    nodes_expanded = 0
    max_depth = 0
    explored_nodes = set()
    initial_state = None
    def __init__(self, initial_state):
        self.explored_nodes.clear()
        self.initial_state = initial_state
        self.frontier = []
    def ancestral_chain(self):
        current = self.solution
        chain = [current]
        while current.parent is not None:
            chain.append(current.parent)
            current = current.parent
        return chain

    def path(self):
        path = [node.operator for node in self.ancestral_chain()[-2::-1]]
        return path

    def set_solution(self, board):
        self.solution = board
        self.nodes_expanded = len(self.explored_nodes) - len(self.frontier) - 1
        # return self.solution
        
    def dls(self, limit):
        while self.frontier:
            board = self.frontier.pop()
            new_board = np.append(board.state, board.depth)
            current_state = tuple(new_board)
            self.explored_nodes.add(current_state)
            np.delete(new_board, -1)
            if board.goal_test():
                self.set_solution(board)
                return self.solution
            if board.depth < limit:
                for neighbor in board.neighbors()[::-1]:
                    new_neighbor = np.append(neighbor.state, neighbor.depth)
                    if tuple(new_neighbor) not in self.explored_nodes:
                        self.frontier.append(neighbor)
                        self.explored_nodes.add(tuple(new_neighbor))
                        self.max_depth = max(self.max_depth, neighbor.depth)
        return None
    
    # same as dfs just has limit iteratively
    def solve(self):
        for i in itertools.count():
            self.frontier = []
            self.explored_nodes = set()
            self.explored_nodes.clear()
            self.max_depth = 0
            self.frontier.append(self.initial_state)
            sol = self.dls(i)
            if sol is not None:
                break
        return

here we read the Examples.txt file

In [8]:
with open('Examples.txt') as f:
    easy_lines = f.readlines()

easy = []
for line in easy_lines:
    easy.append(eval(line))

print(type(easy))

<class 'list'>


now we import libraries and define the records dictionary

In [10]:
import pandas as pd
from contextlib import nullcontext
import resource
import sys
import numpy as np
import time
import tracemalloc

records = {}


In [11]:
bfs_records = pd.DataFrame(columns = ['test num','running time', 'ram usage', 'nodes explored', 'cost of path', 'path'])
for i in range (0,len(easy)):
    p = Board(easy[i])
    s = BFS(p)
    
    tracemalloc.start()

    t0 = time.time()
    s.solve()
    t1 = time.time()

    run_time = t1-t0

    ram_usage = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    cost_path = len(s.path())

    bfs_records.loc[i] = [i, run_time, ram_usage[0], len(s.explored_nodes), cost_path, s.path()]

records['BFS'] = bfs_records
records['BFS']

,test num,running time,ram usage,nodes explored,cost of path,path
0,0,0.066100,390092,371,7,"[Down, Right, Up, Left, Down, Right, Right]"
1,1,0.343242,1677422,1733,10,"[Right, Right, Down, Left, Left, Up, Right, Do..."
2,2,0.027891,89368,105,5,"[Up, Right, Down, Down, Right]"
3,3,0.070217,365560,420,7,"[Right, Down, Right, Up, Left, Down, Right]"
4,4,0.038267,135656,164,6,"[Down, Down, Left, Up, Right, Down]"
5,5,0.022934,117336,143,5,"[Right, Down, Left, Down, Right]"
6,6,0.009918,50088,65,4,"[Right, Right, Down, Down]"
7,7,0.006167,32960,42,3,"[Right, Right, Down]"
8,8,0.003631,20312,25,2,"[Right, Down]"
9,9,0.001464,4568,6,1,[Down]


In [12]:
dfs_records = pd.DataFrame(columns = ['test num','running time', 'ram usage', 'nodes explored', 'cost of path', 'path'])
for i in range (0,len(easy)):
    p = Board(easy[i])
    max_moves = 10
    if(i > 19 and i < 35):
        max_moves = 20
    elif(i >= 35):
        max_moves = 32

    tracemalloc.start()
    s = DFS(p,max_moves)

    t0 = time.time()
    s.solve()
    t1 = time.time()

    run_time = t1-t0
    ram_usage = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    cost_path = len(s.path())
    dfs_records.loc[i] = [i, run_time, ram_usage[0], len(s.explored_nodes), cost_path, s.path()]

records['DFS'] = dfs_records
records['DFS']

,test num,running time,ram usage,nodes explored,cost of path,path
0,0,1.374324,133749,262,9,"[Up, Down, Down, Right, Up, Left, Down, Right,..."
1,1,0.282925,405754,1089,10,"[Right, Right, Down, Left, Left, Up, Right, Do..."
2,2,0.010657,28768,66,9,"[Up, Down, Up, Down, Up, Right, Down, Down, Ri..."
3,3,0.079529,137084,393,9,"[Up, Down, Right, Down, Right, Up, Left, Down,..."
4,4,0.010582,28264,66,10,"[Down, Up, Down, Up, Down, Down, Left, Up, Rig..."
5,5,0.029156,56160,155,9,"[Down, Up, Down, Up, Right, Down, Left, Down, ..."
6,6,0.009045,25744,54,10,"[Down, Up, Down, Up, Down, Up, Right, Right, D..."
7,7,0.013715,25456,57,9,"[Up, Down, Up, Down, Up, Down, Right, Right, D..."
8,8,0.005367,28128,42,10,"[Up, Down, Up, Down, Up, Down, Up, Down, Right..."
9,9,0.002993,19800,26,9,"[Up, Down, Up, Down, Up, Down, Up, Down, Down]"


In [13]:
astar_records = pd.DataFrame(columns = ['test num','running time', 'ram usage', 'nodes explored', 'cost of path', 'path'])
for i in range (0,len(easy)):
    p = Board(easy[i])
    s = AStar(p)

    tracemalloc.start()

    t0 = time.time()
    s.solve()
    t1 = time.time()

    run_time = t1-t0

    ram_usage = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    cost_path = len(s.path())
    astar_records.loc[i] = [i, run_time, ram_usage[0], len(s.explored_nodes), cost_path, s.path()]

records['AStar'] = astar_records
records['AStar']

,test num,running time,ram usage,nodes explored,cost of path,path
0,0,0.007998,38320,37,7,"[Down, Right, Up, Left, Down, Right, Right]"
1,1,0.015285,56650,66,10,"[Right, Right, Down, Left, Left, Up, Right, Do..."
2,2,0.001741,12136,16,5,"[Up, Right, Down, Down, Right]"
3,3,0.004070,28696,36,7,"[Right, Down, Right, Up, Left, Down, Right]"
4,4,0.001913,13624,18,6,"[Down, Down, Left, Up, Right, Down]"
5,5,0.001684,12136,16,5,"[Right, Down, Left, Down, Right]"
6,6,0.001295,8352,11,4,"[Right, Right, Down, Down]"
7,7,0.001161,8592,11,3,"[Right, Right, Down]"
8,8,0.001474,6120,8,2,"[Right, Down]"
9,9,0.000428,2600,4,1,[Down]


In [33]:
ids_records = pd.DataFrame(columns = ['test num','running time', 'ram usage', 'nodes explored', 'cost of path', 'path'])
for i in range (0,len(easy)):
    p = Board(easy[i])
    s = IDS(p)

    tracemalloc.start()

    t0 = time.time()
    s.solve()
    t1 = time.time()

    run_time = t1-t0
    ram_usage = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    cost_path = len(s.path())
    ids_records.loc[i] = [i, run_time,ram_usage[0],len(s.explored_nodes),cost_path, s.path()]

records['IDS'] = ids_records
records['IDS']

,test num,running time,ram usage,nodes explored,cost of path,path
0,0,0.108354,429645,153,7,"[Down, Right, Up, Left, Down, Right, Right]"
1,1,0.509464,386712,1089,10,"[Right, Right, Down, Left, Left, Up, Right, Do..."
2,2,0.021787,22072,35,5,"[Up, Right, Down, Down, Right]"
3,3,0.100684,94608,235,7,"[Right, Down, Right, Up, Left, Down, Right]"
4,4,0.048451,48010,56,6,"[Down, Down, Left, Up, Right, Down]"
5,5,0.033635,36688,82,5,"[Right, Down, Left, Down, Right]"
6,6,0.010409,15072,39,4,"[Right, Right, Down, Down]"
7,7,0.005477,10064,23,3,"[Right, Right, Down]"
8,8,0.002929,5936,14,2,"[Right, Down]"
9,9,0.000524,2544,4,1,[Down]


In [15]:
ucs_records = pd.DataFrame(columns = ['test num','running time', 'ram usage', 'nodes explored', 'cost of path', 'path'])
for i in range (0,len(easy)):
    print(i)
    p = Board(easy[i])
    s = BFS(p)

    tracemalloc.start()

    t0 = time.time()
    s.solve()
    t1 = time.time()

    run_time = t1-t0
    ram_usage = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    cost_path = len(s.path())
    ucs_records.loc[i] = [i, run_time,ram_usage[0],len(s.explored_nodes),cost_path, s.path()]

records['UCS'] = ucs_records
records['UCS']

,test num,running time,ram usage,nodes explored,cost of path,path
0,0,0.066100,390092,371,7,"[Down, Right, Up, Left, Down, Right, Right]"
1,1,0.343242,1677422,1733,10,"[Right, Right, Down, Left, Left, Up, Right, Do..."
2,2,0.027891,89368,105,5,"[Up, Right, Down, Down, Right]"
3,3,0.070217,365560,420,7,"[Right, Down, Right, Up, Left, Down, Right]"
4,4,0.038267,135656,164,6,"[Down, Down, Left, Up, Right, Down]"
5,5,0.022934,117336,143,5,"[Right, Down, Left, Down, Right]"
6,6,0.009918,50088,65,4,"[Right, Right, Down, Down]"
7,7,0.006167,32960,42,3,"[Right, Right, Down]"
8,8,0.003631,20312,25,2,"[Right, Down]"
9,9,0.001464,4568,6,1,[Down]


In [16]:
pip install openpyxl==2.6.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openpyxl: filename=openpyxl-2.6.3-py2.py3-none-any.whl size=245722 sha256=46ef34821657da0393ad22a122cb2f5d4abd75b572fa58a6a9a500c9c7de71a7
  Stored in directory: /root/.cache/pip/wheels/cd/35/57/e510f3a7727b79065ad2fc12c5905c05eb440f4f61f991d0e9
Successfully built openpyxl
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.0.10
    Uninstalling openpyxl-3.0.10:
      Successfully uninstalled openpyxl-3.0.10


In [36]:
import pandas as pd
import openpyxl


filename = 'Results.xlsx'  # must exist

wb = openpyxl.load_workbook(filename)

writer = pd.ExcelWriter(filename, engine='openpyxl')

for sheet, frame in records.items():
    writer.sheets = dict((ws.title, ws) for ws in wb.worksheets) # need this to prevent overwrite
    frame.to_excel(writer, index=False, sheet_name = sheet)

writer.save()

# convert data to tables
wb = openpyxl.load_workbook(filename)
for ws in wb.worksheets:
   mxrow = ws.max_row
   mxcol = ws.max_column
   tab = openpyxl.worksheet.table.Table(displayName=ws.title, ref="A1:" + ws.cell(mxrow,mxcol).coordinate)
   ws.add_table(tab)

wb.save(filename)

In [34]:
print("average of running time in BFS algorithm is:", records['BFS']['running time'].mean())
print("average of running time in DFS algorithm is:", records['DFS']['running time'].mean())
print("average of running time in UCS algorithm is:", records['UCS']['running time'].mean())
print("average of running time in IDS algorithm is:", records['IDS']['running time'].mean())
print("average of running time in A* algorithm is:", records['AStar']['running time'].mean())


average of running time in BFS algorithm is: 44.207189525257455
average of running time in DFS algorithm is: 28.268597017635
average of running time in UCS algorithm is: 44.207189525257455
average of running time in IDS algorithm is: 138.67790994644164
average of running time in A* algorithm is: 0.7727122046730736


In [35]:
print("average of ram usage in BFS algorithm is:", records['BFS']['ram usage'].mean())
print("average of ram usage in DFS algorithm is:", records['DFS']['ram usage'].mean())
print("average of ram usage in UCS algorithm is:", records['UCS']['ram usage'].mean())
print("average of ram usage in IDS algorithm is:", records['IDS']['ram usage'].mean())
print("average of ram usage in A* algorithm is:", records['AStar']['ram usage'].mean())

average of ram usage in BFS algorithm is: 104020884.23636363
average of ram usage in DFS algorithm is: 36650552.69090909
average of ram usage in UCS algorithm is: 104020884.23636363
average of ram usage in IDS algorithm is: 32529459.436363637
average of ram usage in A* algorithm is: 2912707.2
